In [1]:
import os
import glob
import pandas as pd
import numpy as np
import xarray as xr

import cartopy.crs as ccrs
import cartopy.feature as cf

import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec

In [2]:
TIME_REF = pd.date_range(start=f'2024-01-01 00:00', end=f'2024-12-31 23:30', freq='30min')
TIME_PD = pd.DataFrame({'datetime':TIME_REF})
print(f'Number of times: {len(TIME_PD)}')

Number of times: 17568


In [3]:
file_list = glob.glob(os.path.join('/','media','arturo','Arturo','Data','Brazil','IMERG','raw','2024','30min','*'))
file_list = sorted(file_list)
file_len = len(file_list)
print(f'Number of files: {file_len}')

Number of files: 17568


In [4]:
PRE = None
for nn in range(file_len):
    tmp = xr.open_dataset(file_list[nn])
    if PRE is None:
        lat = tmp['lat'].values
        lon = tmp['lon'].values
        PRE = np.zeros([file_len, len(lat), len(lon)])
    
    PRE[nn,:,:] = tmp['precipitation'][0,:,:].values.T

In [5]:
PRE_xr = xr.Dataset(data_vars={"PRE": (("time","lat","lon"), PRE)},
                    coords={'time': TIME_PD['datetime'].values, 'lat': lat, 'lon': lon},
                    attrs=dict(description='IMERG 30 min precipitation 2024 for SA',))

PRE_xr.PRE.attrs["units"] = "mm/h"
PRE_xr.PRE.attrs["long_name"] = "precipitation"
PRE_xr.PRE.attrs["origname"] = "precipitation"

PRE_xr.lat.attrs["units"] = "degrees_north"
PRE_xr.lat.attrs["long_name"] = "Latitude"

PRE_xr.lon.attrs["units"] = "degrees_east"
PRE_xr.lon.attrs["long_name"] = "Longitude"

In [6]:
BR_lon_min, BR_lon_max, BR_lat_min, BR_lat_max = -75,-33,-36,6

In [7]:
PRE_xr = PRE_xr.sel(
    lon=slice(BR_lon_min, BR_lon_max),
    lat=slice(BR_lat_min, BR_lat_max))

In [8]:
PRE_xr

<xarray.Dataset> Size: 25GB
Dimensions:  (time: 17568, lat: 420, lon: 420)
Coordinates:
  * time     (time) datetime64[ns] 141kB 2024-01-01 ... 2024-12-31T23:30:00
  * lat      (lat) float32 2kB -35.95 -35.85 -35.75 -35.65 ... 5.75 5.85 5.95
  * lon      (lon) float32 2kB -74.95 -74.85 -74.75 ... -33.25 -33.15 -33.05
Data variables:
    PRE      (time, lat, lon) float64 25GB 0.0 0.0 0.0 0.0 ... 0.55 0.12 0.21
Attributes:
    description:  IMERG 30 min precipitation 2024 for SA

In [9]:
del PRE

In [ ]:
PRE_out = os.path.join('/','media','arturo','Arturo','Data','Brazil','IMERG','IMERG_BR_30min_2024_01_01_2024_12_31.nc')
print(f'Export PRE data to {PRE_out}')
PRE_xr.to_netcdf(PRE_out)

Export PRE data to /media/arturo/Arturo/Data/Brazil/IMERG/IMERG_BR_30min_2024_01_01_2024_12_31.nc
